# Integração

In [ ]:

def simpson(a,b,N,z):                   # definição de uma função que aplica a regra de simpson a uma função g(),
    h = (b-a)/N                         # retornando um número.
    odds = 0
    evens = 0
    for i in range(1,N,2):
        odds += g(a+i*h,z)
    for i in range(2,N,2):
        evens+=g(a+i*h,z)
    return (1/3)*h*(g(a,z)+g(b,z)+4*odds+2*evens)




## Simpson a duas dimensões

In [ ]:
def simpson1(t,z,N,a,b,n):                      # definição de uma função que aplica a regra de simpson
    h = (b-a)/N                                 # à função integranda na varável r (equivalente à integração de g em r)
    odds = 0
    evens = 0
    for i in range(1,N,2):
        odds += g(a+i*h,t,z,n)
    for i in range(2,N,2):
        evens+=g(a+i*h,t,z,n)
    return (1/3)*h*(g(a,t,z,n)+g(b,t,z,n)+4*odds+2*evens)

def simpson2(z,N,a,b,c,d,n):                  # definição de uma função que aplica a regra de simpson à função
    h = (d-c)/N                               # simpson1() na variável t (theta)
    odds = 0
    evens = 0
    for i in range(1,N,2):
        odds += simpson1(c+i*h,z,N,a,b,n)
    for i in range(2,N,2):
        evens+=simpson1(c+i*h,z,N,a,b,n)
    return (1/3)*h*(simpson1(c,z,N,a,b,n)+simpson1(d,z,N,a,b,n)+4*odds+2*evens)

## Romberg

In [ ]:

def recurso(matriz,i,j,N0,a,b):            # função recursiva que calcula todas as entradas
    if j == 0:                             # de uma linha usando como argumento os indices de uma entrada, 
        N = N0*2**i                        # calculando todas as entradas para trás dessa
        h = (b-a)/N
        sum = 0
        for k in range(1,N,2):
            sum += f(a + k*h)
        matriz[i][j] = 0.5*matriz[i-1][j] + h*sum
    else:
        recurso(matriz,i,j-1,N0,a,b)
        matriz[i][j] = matriz[i][j-1] + (1/(4**(j)-1))*(matriz[i][j-1]-matriz[i-1][j-1])
        

def romberg(matriz,N0,a,b,erro):                     # função que implementa recurso() e pára baseada num erro
    recurso(matriz,1,1,N0,a,b)    
    e = (1/(4**(1)-1))*(matriz[1][0]-matriz[0][0])
    i = 0
    while np.abs(e-erro)>erro:
        i += 1
        matriz[i][i] = matriz[i][i-1] + e
        recurso(matriz,i+1,i,N0,a,b)
        h = (b-a)/(N0*2**i)
        e = (1/(4**(i+1)-1))*(matriz[i+1][i]-matriz[i][i])
    return matriz[i+1][i]

# Equações diferenciais 

## Kutta não adaptativo

In [ ]:
def kuta4(t,x0):
    N = len(t)
    h = (t[-1]-t[0])/N
    x = np.zeros(N)
    x[0] = x0
    for i in range(N-1):
        k1 = h*f(x[i],t[i])
        k2 = h*f(x[i] + k1/2,t[i] + h/2)
        k3 = h*f(x[i] + k2/2,t[i] + h/2)
        k4 = h*f(x[i] + k3,t[i] + h)
        x[i+1]= x[i] + (k1 + 2*k2 + 2*k3 + k4)/6
    return x

def vec_kuta4(t,x0):
    N = len(t)
    h = (t[-1]-t[0])/N
    x = np.zeros( (N,len(x0) ),float)
    x[0] = np.copy(x0)
    for i in range(N-1):
        k1 = np.copy(h*f(x[i],t[i]))
        k2 = np.copy(h*f(x[i] + k1/2,t[i] + h/2))
        k3 = np.copy(h*f(x[i] + k2/2,t[i] + h/2)) 
        k4 = np.copy(h*f(x[i] + k3,t[i] + h))
        x[i+1]= np.copy(x[i] + (k1 + 2*k2 + 2*k3 + k4)/6)
    return x

## Kutta Adaptativo

In [ ]:
def kuta_aux(t,x,h):
    k1 = h*f(x,t)
    k2 = h*f(x + k1/2,t + h/2)
    k3 = h*f(x + k2/2,t + h/2)
    k4 = h*f(x + k3,t + h)
    return x + (k1 + 2*k2 + 2*k3 + k4)/6
    

def adapt_kuta(x0,a,b,e,h):
    h1 = h
    temp = [a]
    x =[x0]
    t = a
    while t < b:
        x1 = x0
        x2 = kuta_aux(t,x1,2*h1)
        for j in range(2):
            x1 = kuta_aux(t + j*h1,x1,h1)
        rho = 30*h1*e/(np.abs(x1-x2))
        if rho > 1.:
            x0 = x1
            t += 2*h1
            temp.append(t)
            x.append(x1)
        h1 = min(h1*rho**0.25,2*h1)
    return np.array([temp,x],float)

def adapt_kuta1(x0,a,b,e,h):
    h1 = h 
    temp = np.zeros(300000)
    x =np.zeros(300000)
    temp[0] = a
    x[0]= x0
    t = a
    i = 0
    while t < b:
        x1 = x0
        x2 = kuta_aux(t,x1,2*h1)
        for j in range(2):
            x1 = kuta_aux(t + j*h1,x1,h1)
        rho = 30*h1*e/(np.abs(x1-x2))
        
        if rho > 1.:
            x0 = x1
            i += 1
            t += 2*h1
            temp[i] = t
            x[i] = x1
        h1 = min(h1*rho**0.25,2*h1)
        
    return np.array([temp[:i+1],x[:i+1]],float)

def vec_adapt_kuta(x0,a,b,e,h):
    h1 = h
    temp = [a]
    x =[float(x0[0])]
    r = np.copy(x0)
    t = a
    while t < b:
        x1 = np.copy(r)
        x2 = np.copy(kuta_aux(t,x1,2*h1))
        for j in range(2):
            x1 = np.copy(kuta_aux(t + j*h1,x1,h1))
        rho = 30*h1*e/(np.abs(x1[0]-x2[0]))
        if rho > 1.:
            r = np.copy(x1)
            t += 2*h1
            temp.append(t)
            x.append(x1[0])
        h1 = min(h1*rho**0.25,2*h1)
    return np.array([temp,x],float)

def vec_adapt_kuta1(x0,a,b,e,h):
    h1 = h
    N = 30000
    temp = np.zeros(N)
    x = np.zeros(N)
    temp[0] = a
    x[0] = float(x0[0])
    r = np.copy(x0)
    t = a
    i = 0
    s = 1
    while t < b:
        x1 = np.copy(r)
        x2 = np.copy(kuta_aux(t,x1,2*h1))
        for j in range(2):
            x1 = np.copy(kuta_aux(t + j*h1,x1,h1))
        rho = 30*h1*e/(np.abs(x1[0]-x2[0]))
        if rho > 1.:
            i += 1
            r = np.copy(x1)
            t += 2*h1
            temp[i] = t
            x[i] = x1[0]
        h1 = min(h1*rho**0.25,2*h1)
        s = min(s,h1)
    print(s)
    return np.array([temp[:i+1],x[:i+1]],float)

## Leap frog

In [ ]:
def frog(r0,t,j = 1):                                # se j = 1, faz runge kutta normal. se j = -1, faz runge kutta 
    N = len(t)                                       # invertido.
    h = j*(t[-1]-t[0])/N
    r = np.zeros((N,len(r0)),float)
    aux = np.zeros((N,len(r0)),float)
    r[min(j,0)] = np.copy(r0)
    aux[min(j,0)] = np.copy(r0 + 0.5*h*f(r0,t[0]))
    
    for i in range(j*min(0,j)*(N-1), max(0,j)*(N-1), j):
        
        r[i+j] = r[i] + h*f(aux[i],t[i] + 1/2*h)
        aux[i+j] = aux[i] + h*f(r[(i+j)], t[i] + h)
    return r 

# Matrizes e sistemas matriciais 

In [ ]:
def QR(A):                                          # decomposição QR
    N = len(A[0])
    U = np.zeros([N,N])
    Q = np.zeros([N,N])
    R = np.zeros([N,N])
    U[0] = A[0]
    Q[0] = U[0] / np.linalg.norm([U[0]])
    for i in range(1,N):
        s = np.zeros(N)
        for j in range(i+1):
            s = s + ( np.dot(Q[j],A[i]) * Q[j] )
        U[i] = A[i] - s
        Q[i] = U[i] / np.linalg.norm(U[i])
    for k in range(N):
        R[k,k] = np.linalg.norm(U[k])
        for l in range(k+1,N):
            R[k,l] = np.dot(Q[k],A[l])
    Q = np.matrix.transpose(Q)
    results = [Q,R]
    return results

def eigen(A,epsilon):                                 #obter valores e vetores próprios a menos de um erro
    N = len(A[0])
    V = np.identity(N)
    values = np.zeros(N)
    trigger = True
    while trigger:
        trigger = False
        Q = QR(A)[0]
        R = QR(A)[1]
        A = np.dot(R,Q)
        V = np.dot(V,Q)
        for i in range(N):
            for j in range(N):
                if A[i,j] > epsilon and i!=j :
                    trigger = True
    for i in range(N):
        values[i] = A[i,i]
    return [values,V]


def thomas(trid,s):                        #eliminação gaussiana para matrizes tridiagonais 
    matriz = np.copy(trid)
    v = np.copy(s)
    for i in range(len(matriz)-1):
        a = matriz[i][i]
        v[i] /= a
        matriz[i] /= a
        b =matriz[i+1][i]
        v[i+1] -= v[i]*b
        matriz[i+1] -= matriz[i]*b
    a = matriz[-1][-1]
    v[-1] /= a
    matriz[-1] /= a
    sol = np.zeros(len(trid))
    for i in range(1,len(trid)+1):
        if i == 0:
            sol[-i] = v[-i]
        else:
            sol[-i] = v[-i] - sol[-i+1]*matriz[-i][-i+1]
    return sol

def matriz(N,a1,a2,b = 0, c = 0):                 # criar matrizes tridiagonais
    if b == 0:
        b = a1
        
    A = np.zeros((N,N),complex)
    for i in range(1,N-1):
        A[i][i] = a1
        A[i][i-1] = a2
        A[i][i+1] = a2
    A[N-1][0] = c
    A[0][N-2] = c    
    A[0][0] = a1
    A[N-1][N-1] = b
    A[0][1] = a2
    A[N-1][N-2] = a2
    return A

import numpy as np

def pivot(matriz,s,i):
    a = 0
    b = 0
    c = np.zeros(len(matriz[0]))
    for j in range(i,len(matriz)):
        if np.abs(matriz[j][i]) > np.abs(a):
            a,b = matriz[j][i],j
    c = np.copy(matriz[i])
    matriz[i] = np.copy(matriz[b])
    matriz[b] = np.copy(c)
    f = np.copy(s[i])
    s[i] = np.copy(s[b])
    s[b] = f
    return matriz


def gauss_piv(matriz,s):                      
    matriz1= np.copy(matriz)
    s1 = np.copy(s)
    for i in range(len(matriz1)):
        pivot(matriz1,s1,i)
        a = matriz1[i][i]
        matriz1[i]=matriz1[i]/a
        s1[i] = s1[i]/a
        for j in range(i + 1,len(matriz1)):
            a = matriz1[j][i]
            matriz1[j] = matriz1[j] - matriz1[i]*a
            s1[j] = s1[j] - s1[i]*a
    s1[-1] /= matriz1[-1][-1]
    matriz1[-1][-1] /= matriz1[-1][-1]
    sol = np.zeros(len(matriz1))
    for i in range(1,len(matriz1)+1):
            sol[-i] = s1[-i] - np.dot(sol,matriz1[-i])
    return sol

def gauss(matriz,s):                       #funções que faz eliminação de gauss em qualquer matriz
    matriz1= np.copy(matriz)
    s1 = np.copy(s)
    for i in range(len(matriz1)):
        a = matriz1[i][i]
        matriz1[i]=matriz1[i]/a
        s1[i] = s1[i]/a
        for j in range(i + 1,len(matriz1)):
            a = matriz1[j][i]
            matriz1[j] = matriz1[j] - matriz1[i]*a
            s1[j] = s1[j] - s1[i]*a
    sol = np.zeros(len(matriz1))
    for i in range(1,len(matriz1)+1):
            sol[-i] = s1[-i] - np.dot(sol,matriz1[-i])
    return sol

def reduc(matriz):                                     # reduz a matriz criando a matriz U
    for i in range(len(matriz)):
        matriz[i]=matriz[i]/matriz[i][i]
        for j in range(i + 1,len(matriz)):
            matriz[j] = matriz[j] - matriz[i]*matriz[j][i]
    return matriz

def lower(matriz,U):                                   # cria matriz L
    N = len(U)
    L = np.zeros((N,N))
    for i in range(len(matriz)):
        for j in range(i+1):
            L[i][j] = matriz[i][j] - np.dot(U[:,j][:j],L[i][:j])
    return L

# Equações não-lineares

## Método da bisseção 

In [ ]:
def bi(x1,x2,error):
    e = 1
    while np.abs(e)>error:
        x = x1 + 0.5*np.abs(x1-x2)
        if f(x)*f(x1)<0:
            x2 = x
        else:
            x1 = x
        e = np.abs(x1-x2)
    x = x1 + 0.5*np.abs(x1-x2)
    return x

## Método de Newton


In [ ]:
def newton(x,error):                     # método de newton sabendo a forma analítica da derivada df(x).
    e = 1
    while np.abs(e)>np.abs(error):
        a,x = x,x-f(x)/df(x)
        e = np.abs(x-a)
    return a

def sec(x1,x2,error):                    # método de newton usando as diferenças finitas para a derivada.
    e = 1
    while np.abs(e)>np.abs(error):
        x1,x3 = x2,x2-f(x2)*(x2-x1)/(f(x2)-f(x1))
        x2 = x3
        e = np.abs(x2-x1)
    return x2

# Transformadas de Fourier

In [ ]:
def dft(y):
    N = len(y)
    c = np.zeros(N,complex)
    for k in range(N//2 + 1):
        for n in range(N):
            c[k] += y[n]*np.exp(-2j*np.pi*k*n/N)
    return c

def fft2(y):
    N = len(y)
    a = np.copy(y)
    p.plot(np.abs(a),'.')
    p.show()
    for m in range(int( log(N,2)) -1 , -1 , -1 ):
        E = np.zeros(len(y),complex)
        for k in range(int(N/(2**m))):
            for i in range(2**m):
                E[i+k*2**m] = a[i + 2**(m+1) * (k%( int(N/(2**(m+1) )))) ] + a[i + 2**m + 2**(m+1) * (k%(int(N/(2**(m+1) ))))]*np.exp(-2j*(2**m)*np.pi*k/N)
        a = np.copy(E)
        p.plot(np.abs(a),'.')
        p.show()
    return a



# Animação 

In [1]:
from matplotlib import animation           
from IPython.display import display, Image
from IPython.display import HTML

def init():
    line.set_data([], [])                     
    return line,

def animate(i):
    line.set_data(x, MA[i])
    return line,

In [ ]:
fig, ax = p.subplots(figsize=(10,5)) 
line, = ax.plot([], []) 
ax.set_ylim(-2,2)
ax.set_xlim(0,x[-1])
fr = len(MA[:,0])
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=fr-1, interval=20,blit=True)
p.close(anim._fig)
HTML(anim.to_html5_video())